In [31]:
import pandas as pd
import re

AADT = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vRXk2ssI2wP32cbxd7gJecku7nS9Mjim7Ed3dorQsgalcBYC6KbxpWKlx0ClBsmsgAcbf5QzQqt2tsy/pub?output=csv")

In [32]:
Car_Crash = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vRJhryMDLGWP2PxsaXiDYb5PdBN_vmZxV0aieOFUJNuD5OBBJTR927qUVRnPFBg_5iDbFgxWzDWPvC9/pub?output=csv")

In [23]:
Car_Crash.head()

,Collision Report Number,Collision Date,County,City,Jurisdiction,Agency,Primary Trafficway,Secondary Trafficway,Block Number,Mile Post,...,Collision Type,Object Struck,Damage Threshold Met,Hit and Run,Motor Vehicles Involved,Passengers Involved,Commercial Carrier Involved,School Bus Involved,Pedestrians Involved,Pedalcyclists Involved
0,EC12134,1/1/2022 12:13:00 AM,King,Bellevue,State Road,State Patrol,WB I 90,FROM SB I-405,NaN,9.72,...,NaN,barrier,Y,N,1,N,N,N,0,0
1,EC09140,1/1/2022 12:26:00 AM,Kitsap,Bremerton,State Road,State Patrol,SR 3,NaN,NaN,36.00,...,NaN,Concrete/Jersey Barrier,Y,N,1,N,N,N,0,0
2,EC28579,1/1/2022 12:33:00 AM,Spokane,Airway Heights,State Road,Municipal/City Police,HWY 2,GARFIELD RD,11900,NaN,...,NaN,NaN,N,N,2,Y,N,N,0,0
3,EC28579,1/1/2022 12:33:00 AM,Spokane,Airway Heights,State Road,Municipal/City Police,HWY 2,GARFIELD RD,11900,NaN,...,NaN,NaN,Y,N,2,Y,N,N,0,0
4,EC09574,1/1/2022 12:48:00 AM,Grant,Mattawa,State Road,State Patrol,SR 243,MORRISON ROAD,NaN,18.00,...,NaN,none,Y,Y,2,Y,N,N,0,0


In [35]:
col1 = []
for x in Car_Crash["Primary Trafficway"]:
    if (not re.match("\D+\d+", x) == None):
        col1.append(re.findall(r'\d+', x)[0])
    else:
        col1.append(None)

In [36]:
Car_Crash = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vRJhryMDLGWP2PxsaXiDYb5PdBN_vmZxV0aieOFUJNuD5OBBJTR927qUVRnPFBg_5iDbFgxWzDWPvC9/pub?output=csv")
Car_Crash.insert(7,"Primary Road Number", col)

In [37]:
Car_Crash.head(25)

,Collision Report Number,Collision Date,County,City,Jurisdiction,Agency,Primary Trafficway,Primary Road Number,Secondary Trafficway,Block Number,...,Collision Type,Object Struck,Damage Threshold Met,Hit and Run,Motor Vehicles Involved,Passengers Involved,Commercial Carrier Involved,School Bus Involved,Pedestrians Involved,Pedalcyclists Involved
0,EC12134,1/1/2022 12:13:00 AM,King,Bellevue,State Road,State Patrol,WB I 90,90,FROM SB I-405,NaN,...,NaN,barrier,Y,N,1,N,N,N,0,0
1,EC09140,1/1/2022 12:26:00 AM,Kitsap,Bremerton,State Road,State Patrol,SR 3,3,NaN,NaN,...,NaN,Concrete/Jersey Barrier,Y,N,1,N,N,N,0,0
2,EC28579,1/1/2022 12:33:00 AM,Spokane,Airway Heights,State Road,Municipal/City Police,HWY 2,2,GARFIELD RD,11900,...,NaN,NaN,N,N,2,Y,N,N,0,0
3,EC28579,1/1/2022 12:33:00 AM,Spokane,Airway Heights,State Road,Municipal/City Police,HWY 2,2,GARFIELD RD,11900,...,NaN,NaN,Y,N,2,Y,N,N,0,0
4,EC09574,1/1/2022 12:48:00 AM,Grant,Mattawa,State Road,State Patrol,SR 243,243,MORRISON ROAD,NaN,...,NaN,none,Y,Y,2,Y,N,N,0,0
5,EC08515,1/1/2022 1:14:00 AM,Snohomish,Monroe,State Road,State Patrol,SR 2,2,SOFIE RD,NaN,...,NaN,NaN,N,N,3,Y,N,N,0,0
6,EC08515,1/1/2022 1:14:00 AM,Snohomish,Monroe,State Road,State Patrol,SR 2,2,SOFIE RD,NaN,...,NaN,NaN,Y,N,3,Y,N,N,0,0
7,EC07461,1/1/2022 1:30:00 AM,Grays Harbor,Hoquiam,State Road,Municipal/City Police,SR 109 BYPASS,109,NaN,NaN,...,NaN,Concrete/Jersey Barrier,Y,N,1,N,N,N,0,0
8,EC32952,1/1/2022 1:37:00 AM,Ferry,Republic,State Road,County Sheriff,HWY 21 N,21,AENEAS CREEK,NaN,...,NaN,NaN,Y,N,2,Y,N,N,0,0
9,EC07367,1/1/2022 1:50:00 AM,Benton,Kennewick,State Road,State Patrol,SR240,240,SR395,NaN,...,NaN,Concrete/Jersey Barrier,Y,N,1,N,N,N,0,0
